# Re-create your own _One Hot Encoder_ 

In [1]:
import pandas as pd
import seaborn as sns

## (1) The Titanic Dataset

In [2]:
# Loading 100% of the dataset. 
# Choose 0.5 to load only 50% of the rows randomly

data = sns.load_dataset('titanic').sample(frac = 1) 
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
675,0,3,male,18.0,0,0,7.7750,S,Third,man,True,NaN,Southampton,no,True
794,0,3,male,25.0,0,0,7.8958,S,Third,man,True,NaN,Southampton,no,True
555,0,1,male,62.0,0,0,26.5500,S,First,man,True,NaN,Southampton,no,True
354,0,3,male,NaN,0,0,7.2250,C,Third,man,True,NaN,Cherbourg,no,True
791,0,2,male,16.0,0,0,26.0000,S,Second,man,True,NaN,Southampton,no,True


In [3]:
from sklearn.model_selection import train_test_split

X = data.drop(columns = ['survived', 'alive', 'who', 'adult_male', 'pclass'])
y = data['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [4]:
X_train

,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
508,male,28.0,0,0,22.5250,S,Third,NaN,Southampton,True
844,male,17.0,0,0,8.6625,S,Third,NaN,Southampton,True
92,male,46.0,1,0,61.1750,S,First,E,Southampton,False
291,female,19.0,1,0,91.0792,C,First,B,Cherbourg,False
678,female,43.0,1,6,46.9000,S,Third,NaN,Southampton,False
...,...,...,...,...,...,...,...,...,...,...
230,female,35.0,1,0,83.4750,S,First,C,Southampton,False
332,male,38.0,0,1,153.4625,S,First,C,Southampton,False
704,male,26.0,1,0,7.8542,S,Third,NaN,Southampton,False
680,female,NaN,0,0,8.1375,Q,Third,NaN,Queenstown,True


## (2) A first pipeline

❓ Create a basic Pipeline which ***encodes categorical features*** and ***scales numerical features*** ❓

💡 Use [`make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html) and [`make_column_transformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_transformer.html)

In [5]:
num_features = ['age','fare','sibsp','parch']
cat_features = ['embarked','class','embark_town']
binary_features = ['sex', 'alone']

In [6]:
# YOUR CODE HERE
from sklearn import set_config
set_config(display="diagram")

from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_transformer = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler()
)

cat_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OneHotEncoder(handle_unknown='ignore')
)

binary_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'), 
    OneHotEncoder(handle_unknown='ignore')
)

preprocessor = make_column_transformer(
    (num_transformer, num_features),
    (cat_transformer, cat_features),
    (binary_transformer, binary_features)
)

preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['age', 'fare', 'sibsp', 'parch']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['embarked', 'class', 'embark_town']),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['sex', 'alone'])])

<details>
    <summary>👩🏻‍🏫 <i>Pipeline</i> vs. <i>make_pipeline</i></summary>

* When you create a Pipeline with `Pipeline()`, you have to:
    - specify all the ***sequential steps of the pipeline*** in a list
    - each step is a tuple with:
        - "name_of_the_step"
        - official Scikit-Learn name of the step
    
```python
Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
```
  
* When you create a Pipeline with `make_pipeline()`,
    - you don't have give a name to each step
    - you can simply chain all the steps together using their official Scikit-Learn name
    - the names of the steps are automatically induced by `make_pipeline`
    
```python
make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler()
)
```
    
</details>

<details>
    <summary>👩🏻‍🏫 <i>ColumnTransformer</i> vs. <i>make_column_transformer</i></summary>

* When you create a ColumnTransformer with `ColumnTransformer()`, you have to:
    - specify all the ***parallel steps of the columns' transformer*** in a list
    - each step is a tuple with:
        - "name_of_the_transformer"
        - the transformer
        - the columns which will be impacted by the transformer
    
```python
ColumnTransformer([
    ('num_transformer', num_transformer, num_features),
    ('cat_transformer', cat_transformer, cat_features)
])
```
  
* When you create a ColumnTransformer with `make_column_transformer()`,
    - you don't have give a name to each parallel step
    - each step is a tuple with:
        - the transformer
        - the columns which will be impacted by the transformer
    
```python
make_column_transformer(
    (num_transformer, num_features),
    (cat_transformer, cat_features)
)
```
    
</details>

❓ Chain this preprocessing pipeline with a classifier and optimize it ❓

In [8]:
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

final_pipe = make_pipeline(
    preprocessor,
    LogisticRegression(penalty='l1', solver='liblinear')
)



search = RandomizedSearchCV(final_pipe,
                            param_distributions={
                                'logisticregression__C': stats.loguniform(0.01,100), 
                            },
                            cv=3, 
                            scoring="accuracy", 
                            n_iter=20, 
                            n_jobs=-1)

search.fit(X_train,y_train);

❓ What are the best params and the best score ❓

In [9]:
# YOUR CODE HERE
search.best_params_

{'logisticregression__C': 43.078754818128346}

In [10]:
search.best_score_

0.8074136008918619

## (3) How could we design a Custom Encoder to keep track of the columns' names?

In [11]:
# By default, OneHotEncoder works with Numpy and loses track of columns' names...
ohe = OneHotEncoder(sparse=False)
ohe.fit_transform(X_train[['sex']])

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [12]:
# ... however, we can access the one-hot-encoded names as follows
ohe.get_feature_names_out()

array(['sex_female', 'sex_male'], dtype=object)

❓ Try to create your own OneHotEncoder so that it preserves the columns names ❓

In [18]:
# YOUR CODE HERE
from sklearn.preprocessing import OneHotEncoder


class CustomOHE1(OneHotEncoder):

    def transform(self, *args, **kwargs):
        return pd.DataFrame(super().transform(*args, **kwargs), columns=self.get_feature_names_out())

In [19]:
c = CustomOHE1(sparse=False)
c.fit(X_train[['sex']])
c.transform(X_train[['sex']])
c.fit_transform(X_train[['sex']]).head()

,sex_female,sex_male
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,1.0,0.0
4,1.0,0.0


🏁 If you want to build a very advanced pipeline, feel free to explore the Optional Challenge dealing the `cars dataset` !

💾 Don't forget to git add/commit/push your notebook.

👏 Congratulations, you are now a master at Pipeline and ColumnTransformer.